In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg_ex').getOrCreate()
from pyspark.ml.classification import LogisticRegression

In [0]:
df = spark.read.format("libsvm").load("/FileStore/tables/sample_libsvm_data.txt")
df.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [0]:
log_reg = LogisticRegression()
fitted_log_reg = log_reg.fit(df)

In [0]:
log_reg_results = fitted_log_reg.summary
log_reg_results.predictions.show(5)

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000486...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716171...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298141...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113070...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [0]:
log_reg_results.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [0]:
lr_train, lr_test = df.randomSplit([0.7,0.3])

In [0]:
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)

In [0]:
prediction_and_labels = fit_final.evaluate(lr_test)
prediction_and_labels.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[16.6785636892646...|[0.99999994290582...|       0.0|
|  0.0|(692,[100,101,102...|[2.80620573857869...|[0.94301025072776...|       0.0|
|  0.0|(692,[122,123,124...|[24.8884898279450...|[0.99999999998447...|       0.0|
|  0.0|(692,[123,124,125...|[39.0984493594645...|           [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[37.8503139056383...|           [1.0,0.0]|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator()

In [0]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)
my_final_roc

Out[11]: 0.9970588235294117

In [0]:
df_titanic = spark.read.table('titanic_1_csv')
df_titanic.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [0]:
df_titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [0]:
df_titanic.columns

Out[16]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
my_cols = df_titanic.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [0]:
my_final_data = my_cols.na.drop()
my_final_data.show(5)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [0]:
gender_indexer = StringIndexer(inputCol = 'Sex',outputCol = 'SexIndex')
gender_encoder = OneHotEncoder(inputCol = 'SexIndex', outputCol = 'SexVec')

In [0]:
embark_indexer = StringIndexer(inputCol = 'Embarked',outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex', outputCol = 'EmbarkVec')

In [0]:
assembler = VectorAssembler(
    inputCols = ['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
    outputCol = 'features'
)

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic = LogisticRegression(featuresCol = 'features',labelCol = 'Survived')

In [0]:
pipeline = Pipeline(
    stages = [
        gender_indexer,
        embark_indexer, 
        gender_encoder, 
        embark_encoder, 
        assembler,
        log_reg_titanic]
)

In [0]:
train_df, test_df = my_final_data.randomSplit([0.7,0.3])

In [0]:
fit_model = pipeline.fit(train_df)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f1969d67b80>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f1969d67b80>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [0]:
results = fit_model.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',labelCol = 'Survived')

In [0]:
results.select('Survived','prediction').show(6)


+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
+--------+----------+
only showing top 6 rows



In [0]:
AUC = my_eval.evaluate(results)
AUC

Out[44]: 0.7804060346433228